In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn.decomposition import PCA
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_parquet('Data/train.parquet')
# test = pd.read_parquet('Data/test.parquet')
train.set_index('customer_ID', inplace=True)
# test.set_index('customer_ID', inplace=True)

columns = train.columns
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
num_columns = [c for c in columns if c not in cat_columns]

train.shape

(5531451, 189)

In [3]:
train.drop(cat_columns, axis=1, inplace=True)
# test.drop(cat_columns, axis=1, inplace=True)

train['time'] = pd.to_datetime(train['S_2'])                                                                                                                                                                                                    
train['time']  = train['time'].astype(np.int64) / 10**18
# test['time'] = pd.to_datetime(test['S_2'])                                                                                                                                                                                                    
# test['time']  = test['time'].astype(np.int64) / 10**18

train.drop(['S_2'], axis=1, inplace=True)
# test.drop(['S_2'], axis=1, inplace=True)

num_columns.remove('S_2')

In [25]:
indx = train[(train.groupby('customer_ID').count()['time'] > 2)].index.unique()

In [70]:
s1_list = []
s2_list = []

for count, id in enumerate(indx):
    train_id = train[train.index == id].copy()
    train_id = train_id.fillna(train_id.median())
    train_pca = PCA(n_components=2).fit_transform(train_id[num_columns])
    train_pca_df = pd.DataFrame(train_pca, columns=['PCA1', 'PCA2'],  index=train_id.index)
    train_pca_df['time'] = train_id['time']
    s1 = train_pca_df.groupby('customer_ID').apply(lambda v: linregress(v['time'], v['PCA1'])[0]).values[0]
    s2 = train_pca_df.groupby('customer_ID').apply(lambda v: linregress(v['time'], v['PCA2'])[0]).values[0]
    s1_list.append(s1)
    s2_list.append(s2)

In [74]:
slope_pca_df = pd.DataFrame(s1_list, index=indx[:5], columns=['s1'])
slope_pca_df['s2'] = s2_list
slope_pca_df

,s1,s2
customer_ID,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,31821.462020,-7258.048778
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,35282.686153,1564.077229
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,69.582284,-55.850760
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,-28216.029327,-197.709235
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,-995.988944,814.943927
